<a href="https://colab.research.google.com/github/KoreTane/Analise-GLP/blob/main/ParquetTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## vamos fazer a conversão de csv para parq#
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=aac1a27c639f396d3255767aa5618129d8f2291ae557577d6ac27d771615440c
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from google.colab import drive

# Monta o Google Drive
drive.mount('/content/drive')

# Inicializa a sessão do Spark
spark = SparkSession.builder \
    .appName("Análise de Preços do GLP") \
    .getOrCreate()

# Lista de arquivos CSV
csv_files = [
    'precos-2022-10-glp.csv',
    'precos-2022-11-glp.csv',
    'precos-2022-12-glp.csv',
    'precosglp2024-08.csv',
    'precosglp2024-07.csv',
    'glp2024-01.csv',
    'glp2023-02.csv',
    'precos-glp-09.csv',
    'glp2023-01.csv'
]

# Dicionário para armazenar os DataFrames
dataframes = {}

# Caminho para salvar os arquivos Parquet no Google Drive
parquet_folder_path = '/content/drive/My Drive/1dhA5CNHrywBWHxOgPRyLalCBmNVbCLof/'  # Ajuste conforme necessário

# Carregar os CSVs com delimitador correto e salvar como Parquet
for csv_file in csv_files:

    df = spark.read.csv(csv_file, header=True, sep=';', inferSchema=True)

    # Salvar como Parquet no Google Drive, com a opção 'overwrite'
    parquet_file_path = f'{parquet_folder_path}{csv_file.split(".")[0]}.parquet'
    df.write.mode('overwrite').parquet(parquet_file_path)

    # Armazenar o DataFrame no dicionário
    dataframes[csv_file] = df

# Mostrar a estrutura e as primeiras linhas de cada DataFrame
for name, df in dataframes.items():
    print(f"Dados de {name}:")
    df.printSchema()
    df.show(10, truncate=False)

# Carregar os arquivos Parquet para realizar a união
parquet_dataframes = {
    f'{csv_file.split(".")[0]}.parquet': spark.read.parquet(f'{parquet_folder_path}{csv_file.split(".")[0]}.parquet')
    for csv_file in csv_files
}


union_df = (
    parquet_dataframes['precos-2022-10-glp.parquet']
    .union(parquet_dataframes['precos-2022-11-glp.parquet'])
    .union(parquet_dataframes['precos-2022-12-glp.parquet'])
    .union(parquet_dataframes['glp2024-01.parquet'])
    .union(parquet_dataframes['glp2023-02.parquet'])
    .union(parquet_dataframes['glp2024-01.parquet'])
    .union(parquet_dataframes['precosglp2024-07.parquet'])
    .union(parquet_dataframes['precosglp2024-08.parquet'])
    .union(parquet_dataframes['precos-glp-09.parquet'])
)


print

Mounted at /content/drive
Dados de precos-2022-10-glp.csv:
root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)

+--------------+--------------+---------+----------------------------------+-------------------+----------------------------+----------+---------------------------+--------------+---------+-------+--------------+--------------+---------------+-----

<function print>